# Data Loading from previous sentiment analysis

In [2]:
import pandas as pd

# Load sentiment stats and sample reviews
stats = pd.read_csv("product_sentiment_stats.csv")
top_reviews = pd.read_csv("top_reviews_per_product.csv")

In [4]:
PROMPT_TEMPLATE = """
Product: {product_name}
Total Reviews: {review_count}
Positive: {pos_pct:.1%}, Negative: {neg_pct:.1%}

Top positive review excerpt:
"{text_pos}"

Top negative review excerpt:
"{text_neg}"

Task: Write a 2–3 sentence summary highlighting:
1. Main strengths
2. Common issues
3. Overall recommendation or verdict.
"""


# Third Party LLM Setup (Mistral)

In [5]:
from huggingface_hub import login

login(token="*")  # paste your Hugging Face token (removed for security)


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.1"  # gated model with access

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto", use_auth_token=True)

generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    temperature=0.7,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


# Using LLM to generate summaries of Product

We see missing reviews for products, we will ommit missing product reviews to generate a summary for products with reviews

In [9]:
missing = set(stats["name"]) - set(reviews["name"])
print("Products missing review entries:", missing)
stats = stats[stats["name"].isin(reviews["name"])]


Products missing review entries: {'Certified Refurbished Amazon Fire TV Stick (Previous Generation - 1st),,,\r\nCertified Refurbished Amazon Fire TV Stick (Previous Generation - 1st),,,', 'Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Free 3G + Wi-Fi - Includes Special Offers', 'Amazon Echo and Fire TV Power Adapter,,,\r\nAmazon Echo and Fire TV Power Adapter,,,', 'Kindle Paperwhite,,,\r\nKindle Paperwhite,,,', 'Amazon Fire TV Gaming Edition Streaming Media Player', 'Cat Litter Box Covered Tray Kitten Extra Large Enclosed Hooded Hidden Toilet', 'AmazonBasics USB 3.0 Cable - A-Male to B-Male - 6 Feet (1.8 Meters)', 'Fire TV Stick Streaming Media Player Pair Kit', 'Echo (Black),,,\r\nEcho (Black),,,', 'Amazon Echo ‚Äì White', 'Fire HD 8 Tablet with Alexa, 8 HD Display, 16 GB, Tangerine - with Special Offers,', 'Amazon - Kindle Voyage - 4GB - Wi-Fi + 3G - Black', 'Echo (Black),,,\r\nAmazon 9W PowerFast Official OEM USB Charger

We make a good prompt which will help maintain consistency and help us generate summaries for the product based of the reviews

In [10]:
import pandas as pd

stats = pd.read_csv("product_sentiment_stats.csv")
reviews = pd.read_csv("top_reviews_per_product.csv")

PROMPT = """
Product: {name}
Reviews: {review_count}, 👍 {pos_pct:.0%}, 👎 {neg_pct:.0%}
Top Positive: "{text_pos}"
Top Negative: "{text_neg}"

Write a concise 1–2 sentence summary: key strength, main issue, and overall recommendation.
"""

summaries = []
for _, r in stats.iterrows():
    revs = reviews[reviews["name"] == r["name"]]
    if revs.empty:
        print(f"⚠️ Skipped '{r['name']}' – no review entry found.")
        continue

    rev = revs.iloc[0]
    prompt = PROMPT.format(
        name=r["name"],
        review_count=r["review_count"],
        pos_pct=r["pos_pct"],
        neg_pct=r["neg_pct"],
        text_pos=rev["reviews.text_pos"],
        text_neg=rev["reviews.text_neg"],
    )
    out = generator(prompt)[0]["generated_text"].strip()
    summaries.append({"name": r["name"], "summary": out})

pd.DataFrame(summaries).to_csv("product_summaries.csv", index=False)
print("✅ Summaries generated and saved.")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


⚠️ Skipped 'All-New Fire HD 8 Tablet, 8" HD Display, Wi-Fi, 32 GB - Includes Special Offers, Magenta' – no review entry found.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


⚠️ Skipped 'All-New Kindle Oasis E-reader - 7 High-Resolution Display (300 ppi), Waterproof, Built-In Audible, 32 GB, Wi-Fi + Free Cellular Connectivity' – no review entry found.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

⚠️ Skipped 'Kindle Paperwhite,,,
Kindle Paperwhite,,,' – no review entry found.
⚠️ Skipped 'Kindle PowerFast International Charging Kit (for accelerated charging in over 200 countries)' – no review entry found.
⚠️ Skipped 'Kindle Voyage E-reader, 6 High-Resolution Display (300 ppi) with Adaptive Built-in Light, PagePress Sensors, Free 3G + Wi-Fi - Includes Special Offers' – no review entry found.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


⚠️ Skipped 'New Amazon Kindle Fire Hd 9w Powerfast Adapter Charger + Micro Usb Angle Cable,,,
' – no review entry found.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


⚠️ Skipped 'Two Door Top Load Pet Kennel Travel Crate Dog Cat Pet Cage Carrier Box Tray 23"' – no review entry found.
✅ Summaries generated and saved.


We now will save these summaries into a csv

In [11]:
pd.DataFrame(summaries).to_csv("product_summaries.csv", index=False)
print("✅ Completed — final summaries saved.")


✅ Completed — final summaries saved.


In [ ]:
%%shell
jupyter nbconvert --to html /content/sentiment_analysis_my.ipynb